In [ ]:
import pandas as pd
from itertools import product
import numpy as np
import pandas as pd
import scipy.integrate as scint
import itertools
import time
from scipy.stats import norm
import math
from lmfit import minimize, Parameters, Parameter, report_fit

def bee_eq(y, t, h, external_temperature, truth, peri, treat):
    hive_temperature = y
    if treat == 0:
        dydt = -h*(w1*w2*(1-np.exp(-alpha*hive_temperature)) / (w2+w1*np.exp(-alpha*hive_temperature)))+ 2*(-hive_temperature + external_temperature) #- np.sum(hive_temperature)
    else:
        dydt = -h*(w1*w2*(1-np.exp(-alpha*hive_temperature)) / (w2+w1*np.exp(-alpha*hive_temperature))) + 2*(-hive_temperature) + peri + external_temperature #- np.sum(hive_temperature)
    return dydt

def run_bee_eq(t, a,h, ext_temp, truth, peri, treat):
    sol = scint.odeint(bee_eq, a, t, args=(h, ext_temp, truth, peri, treat), col_deriv = True, rtol = 1e-3, atol = 1e-3) # 'w' and 'amplitude_temp_ext'
    theta_t = sol[-1,:]
    return theta_t    

def residual(ps, ts, data, treat, positions):
    
    h, theta_ideal = [], []
    k = 0
    h_ = [1]
    l = 96
    for i in range(len(positions)-1):
        data_position = data.groupby(data['Position']) #position -> segment no -> day no
        for s,t in data_position:
            if s == positions[i]:
                left_portion = t
            if s > positions[i] and s < positions[i+1]:
                left_portion = pd.concat([left_portion, t])
        x_left = np.linspace(0, len(left_portion[temp_data].values)-1, num = len(left_portion[temp_data]))
        xx_left =left_portion[ext_data].values
        yy_left = left_portion[temp_data].values
        if type_ == 'Y': #add another column -> 'Treatment' -> 0
            zz_left = left_portion[peri_data].values
            treat = left_portion['Treatment'].values[0]
        else:
            zz_left = left_portion[temp_data].values
            treat = 0
        h = np.concatenate((h,[ps['h_'+str(i)].value]*len(x_left)))

        theta_ideal = np.concatenate((theta_ideal,[ps['theta_ideal_'+str(i)].value]*len(x_left)))
        try:
            h_.append(abs(ps['h_'+str(k)].value - ps['h_'+str(k-1)].value))
        except:
            h_.append(abs(h_[k] - ps['h_'+str(k)].value))
        k = k+1
        
    data['h'] = np.array(h).reshape(-1,1)
    data['theta_ideal'] = np.array(theta_ideal).reshape(-1,1)

    return_value = []
    data_date = pd.DataFrame(data).groupby(data['Date'])
    model = []
    for i,j in data_date:
        ext_temp = j[ext_data].values
        truth = j[temp_data].values
        try:
            treat = j['Treatment'].values[0]
        except:
            treat = 0
        if  treat == 0:
            peri = j[ext_data].values
        else:
            peri = j[peri_data].values
        a = [10]*len(ext_temp)
        fitted = run_bee_eq(ts, a, j['h'].values, ext_temp - j['theta_ideal'].values,truth - j['theta_ideal'].values, peri-j['theta_ideal'].values, treat)
        model = np.concatenate((model,fitted + j['theta_ideal'].values))
        

    return_value = np.concatenate((return_value, (model - data[temp_data]).ravel()))
    return_value = np.concatenate((return_value, l*np.array(h_[2:]).ravel()))
    #print(np.mean(abs((model - data['Temp_'+str(hive_no)]).ravel())))
    return return_value

def residual_2(ps, ts, data, treat):
    #x0 = ps['a0'].value
    
    #w1 = ps['w1'].value
    #w2 = ps['w2'].value
    theta_ideal = ps['theta_ideal'].value
    h = ps['h'].value
    data_date = pd.DataFrame(data).groupby(data['Date'])
    model = []
    for i,j in data_date:
        ext_temp = j[ext_data].values
        truth = j[temp_data].values
        try:
            treat = j['Treatment'].values[0]
        except:
            treat = 0
        if  treat == 0:
            peri = j[ext_data].values
        else:
            peri = j[peri_data].values
        a = [10]*len(ext_temp)
        model = np.concatenate((model,run_bee_eq(ts, a, h, ext_temp - theta_ideal,truth - theta_ideal, peri-theta_ideal, treat)))
    return (model - data[temp_data] + theta_ideal).ravel()
type_ = 'N' #no treatment
filename = '/home/shamima/lm_optimization/C_265-no_hiccups.csv' #filename
temp_data = 'Temp_265' #hive_temperature column name
ext_data = 'Temp_Box' #external temperature column name
#peri_data = 'Temp_Easy_'+ hive_number
peri_data = ext_data
data = pd.read_csv(filename)
params = Parameters()
#params.add('w1', value= 1.0, min=0, max=80)
#params.add('w2', value=1.0, min=0, max=80)
w1 = 80
w2 = 221
#theta_ideal = 35
#params.add('theta_ideal', value = 35.0, min = 31, max = 38)
alpha = 1
d = pd.DataFrame(data).groupby(data['Date'])
h_max = 1

params.add('h', value = np.random.ranf(), min = 0.01, max = h_max)

params.add('theta_ideal', value = 35, min = 31, max = 38) #range of theta_ideal  

x_left = np.linspace(0, len(data[temp_data].values)-1, num = len(data[temp_data]))
xx_left = data[ext_data].values
yy_left = data[temp_data].values
if type_ == 'Y':
    zz_left = data[peri_data].values
    treat = data['Treatment'].values[0]
else:
    zz_left = data[temp_data].values
    treat = 0
result = minimize(residual_2, params, args=(x_left, data, treat), method='leastsq')
sig_left = yy_left + result.residual.reshape(data[temp_data].shape)
L = np.sum(np.log(norm.pdf(data[temp_data].values.reshape(1,-1) - np.array(sig_left).reshape(1,-1))))
pos = [0,len(d)]
b = 1
min_sig = [sig_left]
all_params = [np.array([result.params['h'].value, result.params['theta_ideal'].value])]
all_positions = [pos]
all_lks = [-2*L + 2*2]
#print(all_lks)
while b < len(d):
    print(b)
    position = []
    params_ = []
    signals = []
    likelihood = []
    for a in range(len(d)):
        if a not in pos:
            print(a)
            positions = np.append(pos,a)
            positions.sort()

            sig_left = []
            left_params = []
            
            params = Parameters()
            for i in range(len(positions)-1):
                params.add('h_'+str(i), value = np.random.ranf(), min = 0.01, max = h_max)
            
            for i in range(len(positions)-1):
                params.add('theta_ideal_'+str(i), value = 35, min = 31, max = 38)
                
            result = minimize(residual, params, args=(x_left, data, treat, positions), method='leastsq')
            sig_left = data[temp_data] + result.residual[:len(data[temp_data])].reshape(data[temp_data].shape)
            all_param = []
            for i in range(len(positions)-1):
                all_param.append(result.params['h_'+str(i)].value)
                all_param.append(result.params['theta_ideal_'+str(i)].value)
            left_params.append(np.array(all_param))
            params_.append(left_params)
            #print(params_)
            position.append(positions)
            signals.append(sig_left)
            L = np.sum(np.log(norm.pdf(data[temp_data].values.reshape(1,-1) - np.array(sig_left).reshape(1,-1))))
            likelihood.append(-2*L + 2*((len(positions)-2)+(len(positions)-1)*2))
    predicted = signals[np.argmin(likelihood)]
    all_params.append(params_[np.argmin(likelihood)])
    #print(all_params)
    all_positions.append(position[np.argmin(likelihood)])
    all_lks.append(likelihood[np.argmin(likelihood)])
    min_sig.append(predicted)
    pos = position[np.argmin(likelihood)]
    print(pos) 
    if all_lks[-1] > all_lks[-2]:
        lower = 0.1*min(all_lks)
        for m in range(len(all_lks)):
            if all_lks[m] <= min(all_lks) + lower:
                cut_points = all_positions[m]
                sig = min_sig[m]
                params_ = all_params[m]
                print(cut_points) ##output 1 -> position of the segments
                print(params_) ## output 2 -> parameters for each segments
                for n in sig:
                    print(n) ## output 3 -> reconstructed hive temperature sequences -> (h, theta_ideal)
                break
        break
    b = b+1
    
##data['Reconstructed'] = n